In [2]:
import torch
print("GPU 사용 가능 여부:", torch.cuda.is_available())
print("사용 중인 디바이스:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU")

GPU 사용 가능 여부: True
사용 중인 디바이스: NVIDIA GeForce RTX 4060


### 1. 줄거리 감정분석(plot_sentiment_score)

In [5]:
from transformers import pipeline
import pandas as pd
from tqdm import tqdm
device = 0 if torch.cuda.is_available() else -1  # GPU: 0, CPU: -1

sentiment_model = pipeline(
    "sentiment-analysis",
    model="distilbert-base-uncased-finetuned-sst-2-english",
    device=device
)

Device set to use cuda:0


In [ ]:
df = pd.read_csv("TGDT_per_movie_data_cleaned_plus_feature.csv")

In [ ]:
# 감성 분석 적용 함수
def get_sentiment_score(text):
    try:
        result = sentiment_model(text[:512])[0]
        score = result["score"]
        return score if result["label"] == "POSITIVE" else -score
    except:
        return None


In [ ]:
# 분석 실행
tqdm.pandas()
df["plot_sentiment_score"] = df["overview"].progress_apply(get_sentiment_score)


100%|██████████| 949/949 [00:03<00:00, 273.02it/s]


### 2. 감정 스펙트럼 분류(plot_emotion_*)

In [ ]:
# 1. 라이브러리 불러오기
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import pipeline
import pandas as pd
from tqdm.notebook import tqdm

# 2. 모델 로드
MODEL = "j-hartmann/emotion-english-distilroberta-base"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
emotion_pipe = pipeline("text-classification", model=model, tokenizer=tokenizer, top_k=None)

# 3. 데이터 불러오기
df = pd.read_csv("TGDT_per_movie_data_cleaned_plus_feature.csv")

# 4. 감정 분석 실행
tqdm.pandas()

def extract_emotions(text):
    try:
        results = emotion_pipe(text[:512])[0]  # top_k=None → 모든 감정 점수 반환
        return {f"plot_emotion_{r['label']}": r['score'] for r in results}
    except:
        return {}

# 5. 감정 점수 컬럼 생성
emotion_df = df["overview"].progress_apply(extract_emotions).apply(pd.Series)
df = pd.concat([df, emotion_df], axis=1)

# 6. 결과 저장
df.to_csv("TGDT_per_movie_data_cleaned_plus_feature.csv", index=False, encoding='utf-8-sig')
print("감정 점수 생성 완료: plot_emotion_* ")

tokenizer_config.json:   0%|          | 0.00/294 [00:00<?, ?B/s]

c:\Python 3.13\Lib\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\김지웅\.cache\huggingface\hub\models--j-hartmann--emotion-english-distilroberta-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config.json:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

Device set to use cuda:0


  0%|          | 0/949 [00:00<?, ?it/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/329M [00:00<?, ?B/s]

✔ 감정 점수 생성 완료: plot_emotion_* 컬럼 포함됨


In [15]:
# 6. 결과 저장
df.to_csv("TGDT_per_movie_data_cleaned_plus_feature.csv", index=False, encoding="utf-8-sig")